In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import dst, idst
import sys
sys.path.append("../..") 
import cuqi
from heatmodel import heat

In [ ]:
N = 128
field_type = "KL"   # "KL" or "Step" 

In [ ]:
model = heat(N=N, field_type=field_type)
if field_type=="KL":
    x = model.grid
    true_init = x*np.exp(-2*x)*np.sin(np.pi-x)
else:
    true_par = np.array([3,1,2])
    true_init = model.domain_geometry.par2fun(true_par)

In [ ]:
model.domain_geometry.plot(true_init, is_fun=True)
plt.title("Initial")

In [ ]:
y_obs = model._advance_time(true_init, makeplot=True)

In [ ]:
model.domain_geometry.plot(y_obs, is_fun=True)
plt.title("Observed")

In [ ]:
SNR = 100
sigma = np.linalg.norm(y_obs)/SNR 

In [ ]:
likelihood = cuqi.distribution.Gaussian(model, sigma, np.eye(N))
prior = cuqi.distribution.Gaussian(np.zeros((model.domain_dim,)), 1)
IP = cuqi.problem.BayesianProblem(likelihood, prior, y_obs)

In [ ]:
samples = IP.sample_posterior(5000)

In [ ]:
x_mean = np.mean(samples.samples, axis=-1)

plt.figure()
plt.plot(x_mean)
plt.title("Posterior mean of parameters")

In [ ]:
plt.figure()
model.domain_geometry.plot(x_mean)
model.domain_geometry.plot(true_init, is_fun=True)
plt.legend(["Sample mean","True initial"])
plt.title("Posterior mean in function space")